
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [3]:
geo["CP Ville"] = geo['Code Postal'] + " " + geo["Commune"]

In [4]:
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)

In [5]:
# fonction recherche de ville
def ville(lat, long):
    geo["distance"] = (geo["Latitude"] - lat)**2 + (geo["Longitude"] - long)**2
    return geo.loc[geo.sort_values("distance").index[0]]

In [6]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [7]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

Code Postal                                                     71330
Commune                                                       BOSJEAN
Département                                            SAONE-ET-LOIRE
Région                                                      BOURGOGNE
Statut                                                 Commune simple
Altitude Moyenne                                                  202
Superficie                                                       1862
Population                                                        0.3
geo_point_2d                             46.7696786287, 5.34727924927
geo_shape           {"type": "Polygon", "coordinates": [[[5.345021...
Latitude                                                      46.7697
Longitude                                                     5.34728
CP Ville                                                71330 BOSJEAN
distance                                                  3.15525e-05
Name: 71044, dtype: 

In [8]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

Code Postal                                                     91120
Commune                                                     PALAISEAU
Département                                                   ESSONNE
Région                                                  ILE-DE-FRANCE
Statut                                                Sous-préfecture
Altitude Moyenne                                                  118
Superficie                                                       1168
Population                                                       30.3
geo_point_2d                             48.7146765876, 2.22881488083
geo_shape           {"type": "Polygon", "coordinates": [[[2.259350...
Latitude                                                      48.7147
Longitude                                                     2.22881
CP Ville                                              91120 PALAISEAU
distance                                                  0.000289682
Name: 91477, dtype: 

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [9]:
import json

In [10]:
# On transforme les données de "geo_shape" en dictionnaires
geo["geo_shape"] = geo["geo_shape"].apply(lambda x: json.loads(x))

In [11]:
# On décompose le dictionnaire en deux colonnes séparées
geo["type"] = geo["geo_shape"].apply(lambda x: x["type"])
geo["coordinates"] = geo["geo_shape"].apply(lambda x: x['coordinates'])

In [12]:
#Décompte des valeurs accédées avec la clé "type"
geo["type"].value_counts().sort_index()

MultiPolygon       72
Polygon         36670
Name: type, dtype: int64

In [13]:
#Création d'une colonne contenant la longueur de la liste de coordonnées 
#(nombre de polygones)
geo["nb_polygones"] = geo["coordinates"].apply(lambda x: len(x))

In [14]:
#si longueur des coordonnées = nombre de polygones
geo["nb_polygones"].value_counts().sort_index()

1    36660
2       80
3        1
4        1
Name: nb_polygones, dtype: int64

In [15]:
#Ville la plus complexe en nombre de polygones
geo.loc[geo["nb_polygones"]==max(geo["nb_polygones"])]

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,distance,type,coordinates,nb_polygones
Code INSEE,,,,,,,,,,,,,,,,,
83069,83400,HYERES,VAR,PROVENCE-ALPES-COTE D'AZUR,Chef-lieu canton,60.0,13356.0,54.7,"43.1018713534, 6.18898508469","{'type': 'MultiPolygon', 'coordinates': [[[[6....",43.101871,6.188985,83400 HYERES,47.049422,MultiPolygon,"[[[[6.116721717969871, 43.0347592631124], [6.0...",4


In [16]:
#si longueur des coordonnées = nombre de points
geo["nb_points"] = geo["coordinates"].apply(lambda x: sum([len(x[i]) for i in range(len(x))]))
geo["nb_points"].value_counts().sort_index()

2        70
3         1
4         5
5       149
6       583
7      1326
8      2174
9      2822
       ... 
203       1
226       1
251       1
273       1
325       1
334       1
392       1
486       1
Name: nb_points, Length: 81, dtype: int64

In [17]:
#Ville la plus complexe en nombre de points
geo.loc[geo["nb_points"]==max(geo['nb_points'])]

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,distance,type,coordinates,nb_polygones,nb_points
Code INSEE,,,,,,,,,,,,,,,,,,
97353,97370,MARIPASOULA,GUYANE,GUYANE,Chef-lieu canton,189.0,1871833.0,7.6,"2.97088417594, -53.7584193112","{'type': 'Polygon', 'coordinates': [[[-53.3171...",2.970884,-53.758419,97370 MARIPASOULA,5228.94962,Polygon,"[[[-53.317197766565286, 2.340121079614097], [-...",1,486


In [18]:
#2è Ville la plus complexe en nombre de points
max2 = max(geo.loc[geo["nb_points"] < max(geo['nb_points'])]["nb_points"])
geo.loc[geo["nb_points"]== max2]

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,distance,type,coordinates,nb_polygones,nb_points
Code INSEE,,,,,,,,,,,,,,,,,,
97301,97390,REGINA,GUYANE,GUYANE,Chef-lieu canton,88.0,1156376.0,0.8,"3.94091776652, -52.5199269871","{'type': 'Polygon', 'coordinates': [[[-52.5010...",3.940918,-52.519927,97390 REGINA,5003.957195,Polygon,"[[[-52.501078590007516, 3.49466089258716], [-5...",1,392


In [19]:
#villes de type "Polygon", mais de nb de polygones égal à 2 
df = geo.loc[(geo["type"] == "Polygon") & (geo["nb_polygones"] == 2)].copy()

In [20]:
#Vérifier que le premier polygone contient bien le second (enclave).
import shapely
from shapely import geometry
from shapely.geometry import Polygon

In [22]:
#Extraire les deux polygones, et les transformer en type Polygon
df['poly1']=df['coordinates'].apply(lambda x: Polygon(x[0]))
df['poly2']=df['coordinates'].apply(lambda x: Polygon(x[1]))

In [24]:
#voir si le 1er polynome contient le second
def f(x):
    return x["poly1"].contains(x["poly2"])
df.apply(f, axis=1)

Code INSEE
02232    True
05052    True
09042    True
22209    True
2A272    True
2B049    True
48080    True
67486    True
68078    True
71028    True
dtype: bool